# **Data Cleaning, Manipulation and Analysis**

---

## **Objectives**

The intention of this notebook was tri-fold: data cleaning, data transformation and data loading. Light analysis has also been carried out to better understand, extract and load data.

### **Inputs**

* Dataset retrieved from Kaggle (CSV file containing data regarding patients with, or potentially at risk of, Alzheimer's disease saved to inputs folder)

### **Outputs**

* Data cleaning pipeline (within this notebook)
* Machine learning pipeline (within this notebook)
* Cleaned data (csv file extracted to outputs folder)
* Data for machine learning (txt file extracted to outputs folder)

### **Additional Comments**

* Data was extracted from Kaggle with the source citation included in the README file.
* Data was saved in its raw orginal form and then cleaned (a machine learning dataset with scaling and encoding was also created).

---
---

##### **REMINDER**: 
All notebook cells should be run top-down (you can't create a dynamic where at a given point you need to go back to a previous cell to execute a task and then return to the cell you were working on).

---
---

## **Setup Information**

---
---

#### **IMPORTANT**: 
Before running the cells below, you **MUST** restart the kernel!

**This is because:**
- Windows locks files that are currently in use.
- NumPy is loaded in the current kernel session.
- Restarting clears memory and releases file locks.

**How to restart the kernel:**
1. Click on the restart button above with the circular arrow before it
2. Confirm the restart
3. **Then** run the cells below in order

---
---

### **Change Working Directory**

* When storing the notebooks in a subfolder to run in the editor, for projects such as these, it's best practice to change the working directory. 
* We need to change the working directory from its current folder to its parent folder.

In [ ]:
# Access the current directory with os.getcwd()
import os
current_dir = os.getcwd()
current_dir

'c:\\Users\\F_bee\\Documents\\vs-code\\vs-code-projects\\healthcare-and-public-health\\jupyter_notebooks'

In [ ]:
# Make parent of current directory the new current directory
# Use os.path.dirname() to get parent directory
# Use os.chdir() to define new current directory
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


In [ ]:
# Confirm new current directory
current_dir = os.getcwd()
current_dir

'c:\\Users\\F_bee\\Documents\\vs-code\\vs-code-projects\\healthcare-and-public-health'

---

### **Install Packages**

---

In [ ]:
# Upgrade numpy (run after kernel restart)
%pip install --upgrade numpy

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Install other packages (run after numpy upgrade)
%pip install pandas matplotlib seaborn scikit-learn plotly feature-engine

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Test all imports (run after all packages installed)
import numpy as np
import pandas as pd
import matplotlib as mb
import matplotlib.pyplot as plt
import plotly as pl
import seaborn as sns
import sklearn as sk
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import feature_engine as fe

print("All packages imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Matplotlib version: {mb.__version__}")
print(f"Seaborn version: {sns.__version__}")
print(f"Scikit-learn version: {sk.__version__}")
print(f"Plotly version: {pl.__version__}")
print(f"Feature-engine version: {fe.__version__}")

---

## **Section 1**

### **Data Extraction**
This section contains code for the loading of data.

---

Extract the dataset from the inputs folder and load it to notebook as a DataFrame.

In [ ]:
df = pd.read_csv("inputs/alzheimers_disease_data.csv")
print("Data loaded successfully!")
print(f"DataFrame shape: {df.shape}")
df

Create a random sample of the data. Consider the first 5 rows (head) throughout for better notebook observability.

In [ ]:
df = df.sample(frac=0.25, random_state=10)
print("Data loaded successfully!")
print(f"DataFrame shape: {df.shape}")
print("\nFirst 5 rows:")
df.head()

---

## **Section 2**

### **Data Transformation**
This section contains functions for transformer creation, pipeline code and light analysis.

---

Check the current columns.

In [ ]:
print("Data loaded successfully!")
print("Available columns:")
print(df.columns.tolist())

Check the minimum values for numerical columns.

In [ ]:
print("Data loaded successfully!")
numerical_columns = ["Age", "Gender", "Ethnicity", "EducationLevel", "BMI", "AlcoholConsumption", "PhysicalActivity", "DietQuality", "SleepQuality", "SystolicBP", "DiastolicBP", "CholesterolTotal", "CholesterolLDL", "CholesterolHDL", "CholesterolTriglycerides", "MMSE", "FunctionalAssessment"]
print("Minimum values for numerical columns:")
df[numerical_columns].min()

Check the maximum values for numerical columns.

In [ ]:
print("Data loaded successfully!")
numerical_columns = ["Age", "Gender", "Ethnicity", "EducationLevel", "BMI", "AlcoholConsumption", "PhysicalActivity", "DietQuality", "SleepQuality", "SystolicBP", "DiastolicBP", "CholesterolTotal", "CholesterolLDL", "CholesterolHDL", "CholesterolTriglycerides", "MMSE", "FunctionalAssessment"]
print("Maximum values for numerical columns:")
df[numerical_columns].max()

Check for duplicates and retrieve their sum.

In [ ]:
print("Data loaded successfully!")
df.duplicated().sum()

Check for null values and retrieve their sum.

In [ ]:
print("Data loaded successfully!")
df.isnull().sum()

Create code to populate categorical columns with integer values with their string counterparts.

In [ ]:
print("Data loaded successfully!")
# Replace values in categorical columns for better readability
# Gender mapping
if "Gender" in df.columns:
    df["Gender"] = df["Gender"].replace({0: "Male", 1: "Female"})

# Ethnicity mapping
if "Ethnicity" in df.columns:
    df["Ethnicity"] = df["Ethnicity"].replace({
        0: "Caucasian", 1: "African American", 2: "Asian", 3: "Other"
    })

# Binary columns (0/1 to No/Yes)
binary_cols = ["Smoking", "CardiovascularDisease", "Depression", 
               "MemoryComplaints", "BehavioralProblems", "PersonalityChanges", 
               "DifficultyCompletingTasks"]

for col in binary_cols:
    if col in df.columns:
        df[col] = df[col].replace({0: "No", 1: "Yes"})

# Diagnosis mapping
if "Diagnosis" in df.columns:
    df["Diagnosis"] = df["Diagnosis"].replace({0: "No Dementia", 1: "Dementia"})
print(f"DataFrame shape: {df.shape}")
df.head()

Create functions to load into transformers.

In [ ]:
# Drop specific columns
def drop_columns(df):
    return df.drop(columns=["EducationLevel", "SleepQuality", "FamilyHistoryAlzheimers", "Diabetes", "HeadInjury", "Hypertension", "SystolicBP", "DiastolicBP", "CholesterolLDL", "CholesterolHDL", "CholesterolTriglycerides", "Confusion", "Disorientation", "Forgetfulness", "DoctorInCharge"], errors="ignore")

# Change column locations
def change_column_location(df):
    new_column_order = ["PatientID", "Age", "Gender", "Ethnicity", "BMI", "DietQuality", "PhysicalActivity", "Smoking", "AlcoholConsumption", "CardiovascularDisease", "CholesterolTotal", "FunctionalAssessment", "ADL", "MMSE", "MemoryComplaints", "BehavioralProblems", "PersonalityChanges", "DifficultyCompletingTasks", "Depression", "Diagnosis"]
    # Only include columns that actually exist in the dataframe
    existing_columns = [col for col in new_column_order if col in df.columns]
    return df[existing_columns]

# Convert data types
def convert_data_types(df):
    if "PatientID" in df.columns:
        df["PatientID"] = df["PatientID"].astype(int)
    if "Age" in df.columns:
        df["Age"] = df["Age"].astype(int)
    if "Gender" in df.columns:
        df["Gender"] = df["Gender"].astype(str)
    if "Ethnicity" in df.columns:
        df["Ethnicity"] = df["Ethnicity"].astype(str)
    if "BMI" in df.columns:
        df["BMI"] = df["BMI"].astype(float)
    if "Smoking" in df.columns:
        df["Smoking"] = df["Smoking"].astype(str)
    if "AlcoholConsumption" in df.columns:
        df["AlcoholConsumption"] = df["AlcoholConsumption"].astype(float)
    if "PhysicalActivity" in df.columns:
        df["PhysicalActivity"] = df["PhysicalActivity"].astype(int)
    if "DietQuality" in df.columns:
        df["DietQuality"] = df["DietQuality"].astype(str)
    if "CardiovascularDisease" in df.columns:
        df["CardiovascularDisease"] = df["CardiovascularDisease"].astype(str)
    if "Depression" in df.columns:
        df["Depression"] = df["Depression"].astype(str)
    if "CholesterolTotal" in df.columns:
        df["CholesterolTotal"] = df["CholesterolTotal"].astype(float)
    if "MMSE" in df.columns:
        df["MMSE"] = df["MMSE"].astype(float)
    if "FunctionalAssessment" in df.columns:
        df["FunctionalAssessment"] = df["FunctionalAssessment"].astype(int)
    if "MemoryComplaints" in df.columns:
        df["MemoryComplaints"] = df["MemoryComplaints"].astype(str)
    if "BehavioralProblems" in df.columns:
        df["BehavioralProblems"] = df["BehavioralProblems"].astype(str)
    if "ADL" in df.columns:
        df["ADL"] = df["ADL"].astype(float)
    if "PersonalityChanges" in df.columns:
        df["PersonalityChanges"] = df["PersonalityChanges"].astype(str)
    if "DifficultyCompletingTasks" in df.columns:
        df["DifficultyCompletingTasks"] = df["DifficultyCompletingTasks"].astype(str)
    if "Diagnosis" in df.columns:
        df["Diagnosis"] = df["Diagnosis"].astype(str)
    return df

# Remove outliers using IQR method
def remove_outliers(df):
    columns = ["BMI", "CholesterolTotal"]
    df_cleaned = df.copy()
    for col in columns:
        if col in df_cleaned.columns: 
            Q1 = df_cleaned[col].quantile(0.25)
            Q3 = df_cleaned[col].quantile(0.75)
            IQR = Q3 - Q1
            mask = (df_cleaned[col] >= Q1 - 1.5 * IQR) & (df_cleaned[col] <= Q3 + 1.5 * IQR)
            df_cleaned = df_cleaned[mask]  
    return df_cleaned

# Scale numerical values and encode categorical values
scaling_transformer = ColumnTransformer([
    ("num", StandardScaler(), ["Patient_Age", "BMI", "Alcohol_Consumption", "Physical_Activity", "Cholesterol_Total", "MMSE", "Functional_Assessment", "Activities_Of_Daily_Living"]), 
    ("cat", OneHotEncoder(drop="first", handle_unknown="ignore"), ["Gender", "Ethnicity", "Smoking", "Cardiovascular_Disease", "Depression", "Memory_Complaints", "Behavioral_Problems", "Personality_Changes", "Difficulty_Completing_Tasks"])  
])

# Rename columns
def rename_columns(df):
    return df.rename(columns={
        "PatientID": "Patient_ID",
        "Age": "Patient_Age",
        "AlcoholConsumption": "Alcohol_Consumption",
        "PhysicalActivity": "Physical_Activity",
        "DietQuality": "Diet_Quality",
        "CardiovascularDisease": "Cardiovascular_Disease",
        "CholesterolTotal": "Cholesterol_Total",
        "FunctionalAssessment": "Functional_Assessment",
        "MemoryComplaints": "Memory_Complaints",
        "BehavioralProblems": "Behavioral_Problems",
        "ADL": "Activities_Of_Daily_Living",
        "PersonalityChanges": "Personality_Changes",
        "DifficultyCompletingTasks": "Difficulty_Completing_Tasks",  
    })

# Drop missing values
def drop_missing_values(df):
    return df.dropna()

# Remove duplicates
def remove_duplicates(df):
    return df.drop_duplicates()

# Round numerical values to 2 decimal places
def round_values(df):
    return df.round(2)

# Capitalize column names with proper acronym handling
def capitalize_columns(df):
    def smart_title(text):
        # Common acronyms that should stay uppercase
        acronyms = {
            "bmi": "BMI",
            "mmse": "MMSE", 
            "adl": "ADL",
            "id": "ID"
        }
        
        # Split by underscore and process each part
        parts = text.split("_")
        result_parts = []
        
        for part in parts:
            lower_part = part.lower()
            if lower_part in acronyms:
                result_parts.append(acronyms[lower_part])
            else:
                result_parts.append(part.title())
        
        return "_".join(result_parts)
    
    df.columns = [smart_title(col) for col in df.columns]
    return df

Create the transformers.

In [ ]:
# Define transformers
change_column_location_transformer = FunctionTransformer(change_column_location)
drop_columns_transformer = FunctionTransformer(drop_columns)
convert_data_types_transformer = FunctionTransformer(convert_data_types)
remove_outliers_transformer = FunctionTransformer(remove_outliers)
rename_columns_transformer = FunctionTransformer(rename_columns)
capitalize_columns_transformer = FunctionTransformer(capitalize_columns)
drop_missing_values_transformer = FunctionTransformer(drop_missing_values)
remove_duplicates_transformer = FunctionTransformer(remove_duplicates)
round_values_transformer = FunctionTransformer(round_values)

Create the pipeline.

In [ ]:
# Create data cleaning pipeline
data_cleaning_pipeline = Pipeline([
    ("drop_columns", drop_columns_transformer),
    ("change_column_order", change_column_location_transformer),
    ("convert_data_types", convert_data_types_transformer),
    ("rename_columns", rename_columns_transformer),
    ("capitalize_columns", capitalize_columns_transformer),
    ("remove_outliers", remove_outliers_transformer),
    ("drop_missing_values", drop_missing_values_transformer),
    ("remove_duplicates", remove_duplicates_transformer),
    ("round_values", round_values_transformer)
])

Create advanced machine learning pipeline.

In [ ]:
# Create advanced pipeline with scaling and encoding for machine learning
# This pipeline should clean and preprocess data, rename columns, scale numerical features, encode categorical features and handle unknown categories
data_cleaning_with_ml_pipeline = Pipeline([
    ("drop_columns", drop_columns_transformer),
    ("change_column_order", change_column_location_transformer),
    ("convert_data_types", convert_data_types_transformer),
    ("rename_columns", rename_columns_transformer),
    ("capitalize_columns", capitalize_columns_transformer),
    ("remove_outliers", remove_outliers_transformer),
    ("drop_missing_values", drop_missing_values_transformer),
    ("remove_duplicates", remove_duplicates_transformer),
    ("round_values", round_values_transformer),  
    ("scale_and_encode", scaling_transformer)
])

---

## **Section 3**

### **Data Loading** 
In this section, we fit both pipelines to two separate instances of the same dataframe, allowing for the transformation process to take place, and the creation of new datasets due for loading as new, cleaned CSV and txt documents.

---

Fit the pipeline to the DataFrame.

In [ ]:
# Apply the pipeline to original dataframe
processed_df = data_cleaning_pipeline.fit_transform(df)
print("Data loaded successfully!")
print(f"Processed data shape: {processed_df.shape}")
print(processed_df.head())

Check the current column list after fitting pipeline.

In [ ]:
print("Data loaded successfully!")
print("Processed DataFrame columns:", processed_df.columns.tolist())

fit the machine learning pipeline.

In [ ]:
# Apply the ML pipeline to original dataframe
scaled_encoded_df = data_cleaning_with_ml_pipeline.fit_transform(df)
print("Data loaded successfully!")
print(f"Scaled data shape: {scaled_encoded_df.shape}")
print(scaled_encoded_df)

Load both previously created dataframes to separate CSV files.

In [ ]:
# Save the processed datasets
processed_df.to_csv("outputs/processed_alzheimers_disease_data_unscaled_and_unencoded.csv", index=False)
np.savetxt("outputs/processed_alzheimers_disease_data_scaled_and_encoded.csv", 
           scaled_encoded_df, delimiter=",", fmt="%.6f")
print("Files saved to outputs folder!")

---

## **Conclusion**
The process approached with some difficulty, but in the end, we managed to generate the instances of the datasets we were after. These will then be used within our application.

---

### **Notes** 
**Method**
- Created the mapping and binary columns as the original data was populated with numerical values for all columns.
- Dropped several columns as aim was to focus on partiuclar parameters (health and lifestyle) and to afford a more simplistic, less technical, viewer/user friendly application.
- Also extracted a fractioned/sampled DataFrame from the original at a random state for analysis purposes.

**Issues**
- Following package conflicts with packages like NumPy and Pandas the installation block was added as a precautionary measure.
- Needed to install Jupyter dependencies within the notebook, as kernel kept dying; Python kernel was restarted, then the necessary packages were downloaded.
- Pandas faced issues such as import errors and HTML errors; this was resolved via a Pandas update as well as using the print function. 

**Further Considerations**
- Consider not to rule out further factors, such head injury, other potential comordid diseases such as diabetes, or family history (these were included in the orginal dataset).
- Capitalize, drop missing values and remove duplicates were added for quality assurance purposes.